In [ ]:
import os
import json
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import chromadb
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Load FAQ data
with open('faq.json', 'r') as f:
    data = json.load(f)

documents = [f"Q: {item['question']}\nA: {item['answer']}" for item in data]
metadatas = [{"question": item["question"]} for item in data]

# Generate vector embeddings
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(documents, show_progress_bar=True)

# Setup ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_faq")
collection = chroma_client.get_or_create_collection(name="faq")

if len(collection.get()['ids']) == 0:
    collection.add(
        documents=documents,
        embeddings=embeddings.tolist(),
        metadatas=metadatas,
        ids=[str(i) for i in range(len(documents))]
    )

# Initialize Gemini model
model = genai.GenerativeModel("gemini-1.5-flash")
chat = model.start_chat(history=[])

def generate_gemini_answer(query, k=3, similarity_threshold=0.6):
    try:
        query_embedding = embedder.encode([query])[0]
        results = collection.query(query_embeddings=[query_embedding], n_results=10)

        if not results.get("documents") or not results["documents"][0]:
            return "I'm sorry, I couldn't find any relevant information to answer your question."

        # Filter based on similarity threshold
        docs_scores = zip(results["documents"][0], results["distances"][0])
        filtered = [(doc, score) for doc, score in docs_scores if score < (1 - similarity_threshold)]

        if not filtered:
            return "I'm sorry, I couldn't find any relevant information to answer your question."

        context = "\n\n".join([doc for doc, _ in filtered[:k]])

        prompt = f"""Answer the following question based on the context provided.

Context:
{context}

Question: {query}
Answer:"""

        response = model.generate_content(prompt)
        return response.text

    except Exception as e:
        return f"An error occurred while generating the answer: {str(e)}"

# CHATBOT STARTS HERE

chat_history_log = []

print("🤖 Integrated Gemini Chatbot with RAG (type 'exit' to quit)")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Bot: Goodbye!")
        break

    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    chat_history_log.append(f"{timestamp} | User: {user_input}")

    rag_response = generate_gemini_answer(user_input)
    if rag_response:
        print("Bot (RAG):", rag_response)
        chat_history_log.append(f"{timestamp} | Bot (RAG): {rag_response}")
    else:
        response = chat.send_message(user_input)
        print("Bot:", response.text)
        chat_history_log.append(f"{timestamp} | Bot: {response.text}")

# Save chat history
with open("chat_history.txt", "w", encoding="utf-8") as f:
    for line in chat_history_log:
        f.write(line + "\n")

# Summarize and save the conversation
with open("chat_history.txt", "r", encoding="utf-8") as f:
    chat_text = f.read()

summary_prompt = f"Summarize the following conversation briefly:\n\n{chat_text}"
summary_response = model.generate_content(summary_prompt)
print("\nSummary:\n", summary_response.text)

summary_filename = f"summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
with open(summary_filename, "w", encoding="utf-8") as f:
    f.write(summary_response.text)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🤖 Integrated Gemini Chatbot with RAG (type 'exit' to quit)
Bot (RAG): Hi there! How can I help you today?

Bot (RAG): The provided text focuses on shipping and delivery options.  There is no information about singing capabilities.  Therefore, the answer is **no**.

Bot (RAG): We offer standard, expedited, and express shipping through FedEx, UPS, and USPS.  Free standard shipping is available on orders over $50 within the continental U.S.  However, some items (batteries, aerosols, perishables) may have shipping restrictions.

Bot (RAG): No, you cannot ship children.  Shipping children is illegal and incredibly dangerous.

Bot (RAG): Based on the provided text, you can ship most items, but there are restrictions on batteries, aerosols, and perishables.  The exact restrictions aren't detailed.  To know for sure if a specific item is shippable, you would need to check with the company directly or review their shipping policy.

Bot (RAG): You're welcome!  Is there anything else I can help y